In [1]:
# Import necessary libraries
import warnings
from dotenv import load_dotenv
import pdfplumber
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain import hub
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain
from langchain_groq import ChatGroq

In [2]:
# Load environment variables
load_dotenv()

True

In [3]:
# Suppress warnings
warnings.filterwarnings("ignore")

In [4]:
# Step 1: PDF Extraction
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file using pdfplumber.
    Returns a list of Document objects.
    """
    extracted_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                extracted_text += page_text + "\n"
    return [Document(page_content=extracted_text, metadata={"source": pdf_path})]

# Load the general PDF with extracted text
general_docs = extract_text_from_pdf('/Users/sayo/personal_projects/Usafe_bot/data/general_one.pdf')

In [5]:
# Structured data definitions
hate_crime_definitions = "A hate crime (also known as a bias crime) is a crime where a perpetrator targets a victim due to their physical appearance or perceived membership in a specific social group."
psychological_effects = "Hate crimes have severe psychological impacts. Individuals may experience trauma, depression, and low self-esteem, while targeted groups often feel increased fear and vulnerability"
history_of_hate_crimes = "The term “hate crime” gained common usage in the U.S. during the 1980s, although similar crimes have historical roots"
motivation_behind_hate_crimes = " include thrill-seeking (committed for excitement, often by groups targeting vulnerable individuals), defensive motives (protecting one’s community from perceived threats)"
german_laws = "The German Criminal Code includes several sections addressing hate crimes"
local_resources = "Germany offers numerous resources to support victims of hate crimes"
steps_to_report_a_crime = "To report a hate crime, document the incident, gather evidence, prepare for language barriers, contact local law enforcement, and reach out to support organizations."

# Create structured data documents
structured_data_docs = [
    Document(page_content=hate_crime_definitions, metadata={"source": "definitions"}),
    Document(page_content=psychological_effects, metadata={"source": "psychological_effects"}),
    Document(page_content=history_of_hate_crimes, metadata={"source": "history"}),
    Document(page_content=motivation_behind_hate_crimes, metadata={"source": "motivation"}),
    Document(page_content=german_laws, metadata={"source": "laws"}),
    Document(page_content=local_resources, metadata={"source": "resources"}),
    Document(page_content=steps_to_report_a_crime, metadata={"source": "reporting_steps"})
]

In [6]:
# Step 2: Document Chunking
def chunk_documents(documents, chunk_size=512, chunk_overlap=20):
    """
    Splits documents into smaller chunks for embedding.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_documents(documents=documents)

In [7]:
# Step 2: Chunk Documents
general_chunks = chunk_documents(general_docs)
print(f"Chunks created from general PDF: {len(general_chunks)}")

# Chunk the structured data documents
structured_chunks = chunk_documents(structured_data_docs)
print(f"Chunks created from structured data: {len(structured_chunks)}")

# Combine general chunks with structured data chunks
general_chunks += structured_chunks
print(f"Total chunks after adding structured data: {len(general_chunks)}")

Chunks created from general PDF: 20
Chunks created from structured data: 7
Total chunks after adding structured data: 27


In [8]:
# Step 3: Embedding and Vector Store Creation
def create_vector_store(general_chunks, data_base_name='usafe_general'):
    """
    Creates a vector store using HuggingFace embeddings and saves it locally.
    """
    embeddings_model = HuggingFaceEmbeddings(
        model_name='sentence-transformers/all-mpnet-base-v2'
    )
    vector_store = FAISS.from_documents(general_chunks, embedding=embeddings_model)
    vector_store.save_local(f"./vector_databases/{data_base_name}")

In [9]:
# Create the vector store specifically for the general information
create_vector_store(general_chunks, data_base_name='usafe_general')

In [10]:
# Step 4: Vector Store Retrieval
def load_vector_store(data_base_name='./vector_databases/usafe_general'):
    """
    Loads a vector store from a specified path and returns a retriever.
    """
    embeddings_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
    vector_store = FAISS.load_local(folder_path=data_base_name, embeddings=embeddings_model, allow_dangerous_deserialization=True)
    return vector_store.as_retriever(search_kwargs={'k': 5})

In [11]:
# Initialize retriever for general information
general_info_retriever = load_vector_store('./vector_databases/usafe_general')

In [12]:
type(general_info_retriever)

langchain_core.vectorstores.base.VectorStoreRetriever

In [13]:
def connect_chains(retriever):
    """
    this function connects stuff_documents_chain with retrieval_chain
    """
    stuff_documents_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat")
    )
    retrieval_chain = create_retrieval_chain(
        retriever=retriever,
        combine_docs_chain=stuff_documents_chain
    )
    return retrieval_chain

In [14]:
# Step 5: LLM and Chain Connection
def initialize_llm(model_name="llama3-8b-8192"):
    """
    Initializes the LLM model with specified configurations.
    """
    return ChatGroq(
        model=model_name,
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2
    )

In [15]:
# Load LLM
llm = initialize_llm()

def setup_retrieval_chain(retriever):
    """
    Sets up the document chain and retrieval chain.
    """
    stuff_chain = create_stuff_documents_chain(
        llm=llm,
        prompt=hub.pull("langchain-ai/retrieval-qa-chat")
    )
    return create_retrieval_chain(retriever=retriever, combine_docs_chain=stuff_chain)

In [16]:
usafe_retrieval_chain = setup_retrieval_chain(general_info_retriever)

In [17]:
def query_usafe_bot(user_input, retrieval_chain=usafe_retrieval_chain):
    """
    Queries the Usafe ChatBot with a user's input and returns the response.
    """
    # Query the retrieval chain
    response = retrieval_chain.invoke({"input": user_input})
    
    # Print the response for debugging
    print(f"Response:\n{response['answer'].strip()}")
    
    # Return the response for further use
    return response['answer']

In [18]:
query_usafe_bot("What are the steps to report a hate crime? and what local resources are available in Germany?")

Response:
According to the context, the steps to report a hate crime are:

1. Bring someone who can assist with translation when reporting the incident at a police station.
2. Visit Your Local Police Station: Bring all collected documentation and explain the details to the officer, emphasizing that it was a hate crime. Request a case reference number for future follow-ups.
3. Report Online: If unable to visit a police station, you can file a report online via local authorities' websites, such as the Berlin Police Online Reporting Portal.

As for local resources available in Germany, the context mentions the following:

1. Online Strafanzeige platform: allows individuals to file criminal complaints online, including hate crimes.
2. Meldestelle Respect!: offers a platform to report hate speech and receive expert analysis.
3. Antidiskriminierungsstelle des Bundes (Federal Anti-Discrimination Agency): provides information and support for victims of hate crimes.
4. Get The Trolls Out!: addr

"According to the context, the steps to report a hate crime are:\n\n1. Bring someone who can assist with translation when reporting the incident at a police station.\n2. Visit Your Local Police Station: Bring all collected documentation and explain the details to the officer, emphasizing that it was a hate crime. Request a case reference number for future follow-ups.\n3. Report Online: If unable to visit a police station, you can file a report online via local authorities' websites, such as the Berlin Police Online Reporting Portal.\n\nAs for local resources available in Germany, the context mentions the following:\n\n1. Online Strafanzeige platform: allows individuals to file criminal complaints online, including hate crimes.\n2. Meldestelle Respect!: offers a platform to report hate speech and receive expert analysis.\n3. Antidiskriminierungsstelle des Bundes (Federal Anti-Discrimination Agency): provides information and support for victims of hate crimes.\n4. Get The Trolls Out!: ad

In [19]:
query_usafe_bot("What is the definition of a hate crime?")

Response:
According to the provided context, a hate crime (also known as a bias crime) is a crime where a perpetrator targets a victim due to their physical appearance or perceived membership in a specific social group.


'According to the provided context, a hate crime (also known as a bias crime) is a crime where a perpetrator targets a victim due to their physical appearance or perceived membership in a specific social group.'

In [20]:
query_usafe_bot("What are the steps i can follow to get resources to help me feel better after a hate crime?")

Response:
Based on the context, here are the steps you can follow to get resources to help you feel better after a hate crime:

1. Document the incident: Record as much information as possible about the incident, including dates, times, locations, and details of what happened.
2. Gather evidence: Collect any physical evidence, such as photos, videos, or witness statements, that can help support your report.
3. Prepare for language barriers: If you're not fluent in German, be prepared to seek translation support from organizations like ReachOut Berlin.
4. Contact local law enforcement: Report the incident to the police and provide them with the information and evidence you've gathered.
5. Reach out to support organizations: Contact organizations like the Agency (antidiskriminierungsstelle.de) for counseling and support, and You Are Not Alone (youarenotalone.ai) for mental health resources and coping strategies.
6. Seek additional support: If needed, reach out to organizations like Reach

"Based on the context, here are the steps you can follow to get resources to help you feel better after a hate crime:\n\n1. Document the incident: Record as much information as possible about the incident, including dates, times, locations, and details of what happened.\n2. Gather evidence: Collect any physical evidence, such as photos, videos, or witness statements, that can help support your report.\n3. Prepare for language barriers: If you're not fluent in German, be prepared to seek translation support from organizations like ReachOut Berlin.\n4. Contact local law enforcement: Report the incident to the police and provide them with the information and evidence you've gathered.\n5. Reach out to support organizations: Contact organizations like the Agency (antidiskriminierungsstelle.de) for counseling and support, and You Are Not Alone (youarenotalone.ai) for mental health resources and coping strategies.\n6. Seek additional support: If needed, reach out to organizations like ReachOu

In [21]:
query_usafe_bot("I faced a hate crime and I need help with what to do next.")

Response:
I'm so sorry to hear that you've experienced a hate crime. It's important to prioritize your safety and well-being during this time. Here are some steps you can take:

1. Document the incident: Write down as many details as you can remember about the incident, including what happened, when it happened, and where it happened. Take photos or gather any physical evidence.
2. Gather evidence: Collect any physical evidence that may be relevant to the incident, such as witness statements, videos, or audio recordings.
3. Prepare for language barriers: If you're not fluent in the language spoken by the police or other authorities, consider bringing someone who can assist with translation.
4. Report the incident: You can report the incident to the police at a station or online through the local authorities' reporting platform. Make sure to explain that it was a hate crime and request a case reference number.
5. Seek support: Reach out to organizations that provide support for hate cri

"I'm so sorry to hear that you've experienced a hate crime. It's important to prioritize your safety and well-being during this time. Here are some steps you can take:\n\n1. Document the incident: Write down as many details as you can remember about the incident, including what happened, when it happened, and where it happened. Take photos or gather any physical evidence.\n2. Gather evidence: Collect any physical evidence that may be relevant to the incident, such as witness statements, videos, or audio recordings.\n3. Prepare for language barriers: If you're not fluent in the language spoken by the police or other authorities, consider bringing someone who can assist with translation.\n4. Report the incident: You can report the incident to the police at a station or online through the local authorities' reporting platform. Make sure to explain that it was a hate crime and request a case reference number.\n5. Seek support: Reach out to organizations that provide support for hate crime 

In [22]:
query_usafe_bot("what are the psychological effects of a hate crime on the victim?")

Response:
According to the context, the psychological effects of a hate crime on the victim may include:

1. Trauma
2. Depression
3. Low self-esteem
4. Increased fear and vulnerability


'According to the context, the psychological effects of a hate crime on the victim may include:\n\n1. Trauma\n2. Depression\n3. Low self-esteem\n4. Increased fear and vulnerability'

In [23]:
query_usafe_bot("What are the motivations behind hate crimes?")

Response:
According to the context, the primary motives behind hate crimes, as identified by sociologists like Jack McDevitt and Jack Levin, include:

1. Thrill-seeking (committed for excitement, often by groups targeting vulnerable individuals)
2. Defensive motives (protecting one's community from perceived threats)
3. Retaliatory motives (revenge for perceived offenses)
4. Mission-oriented crimes (ideologically motivated, often targeting symbolically significant sites)

Additionally, the Self-Control Theory suggests that social, cultural, and individual factors contribute to these biases, and offenders often develop biases through social interactions and cultural influences.


"According to the context, the primary motives behind hate crimes, as identified by sociologists like Jack McDevitt and Jack Levin, include:\n\n1. Thrill-seeking (committed for excitement, often by groups targeting vulnerable individuals)\n2. Defensive motives (protecting one's community from perceived threats)\n3. Retaliatory motives (revenge for perceived offenses)\n4. Mission-oriented crimes (ideologically motivated, often targeting symbolically significant sites)\n\nAdditionally, the Self-Control Theory suggests that social, cultural, and individual factors contribute to these biases, and offenders often develop biases through social interactions and cultural influences."